# Wave Equation



In [ ]:
using Revise
using SummationByPartsOperators, OrdinaryDiffEq
using Plots, LaTeXStrings, Printf

# general parameters
xmin = -1.
xmax = +1.
tspan = (0., 8.0)
u0func(x) = exp(-20x^2)
du0func(x) = zero(x)
# homogeneous Neumann and Dirichlet BCs are implemented up to now
left_bc  = Val(:HomogeneousDirichlet)
right_bc = Val(:HomogeneousDirichlet)

# discretisation parameters
interior_order = 4
N = 101

# setup spatial semidiscretisation
D2 = derivative_operator(MattssonSvärdShoeybi2008(), 2, interior_order, xmin, xmax, N)
semidisc = WaveEquationNonperiodicSemidiscretisation(D2, left_bc, right_bc)
ode = semidiscretise(du0func, u0func, semidisc, tspan)

# solve ode
sol = solve(ode, DPRKN6(), dt=0.25*D2.Δx, adaptive=false, 
            saveat=range(first(tspan), stop=last(tspan), length=200))

# visualise the result
plot(xguide=L"x")
plot!(evaluate_coefficients(sol[end].x[2], semidisc), label=L"u")
plot!(evaluate_coefficients(sol[end].x[1], semidisc), label=L"\partial_t u")

In [ ]:
# make a movie
anim = Animation()
idx = 1
x, u = evaluate_coefficients(sol[end].x[2], semidisc)

fig = plot(x, u, xguide=L"x", yguide=L"u", xlim=extrema(x), ylim=(-1., 1.),
           #size=(1024,768), dpi=250,
           label="", title=@sprintf("\$t = %6.2f \$", sol.t[idx]))
for idx in 1:length(sol.t)
    fig[1] = x, sol.u[idx].x[2]
    plot!(title=@sprintf("\$t = %6.2f \$", sol.t[idx]))
    frame(anim)
end
gif(anim)